In [ ]:
import numpy as np
import pandas as pd
import os, sys
from pathlib import Path
from hist import intervals

sys.path.append("/users/hep/eigen1907/Workspace/Workspace-RPC/modules")
from NanoAODTnP.Plotting.DataLoader import DataLoader
from NanoAODTnP.Plotting.Hist1D import init_figure

: 

In [ ]:
'''
def get_effs(data):
    print("Call get_effs()")
    tree = data.tree
    roll_names = np.unique(tree['roll_name'])
    effs = []
    for roll_name in roll_names:
        print(f'append {roll_name} eff')
        roll_mask = (tree['roll_name'] == roll_name)
        total = tree['is_matched'][roll_mask]
        passed = total[total]

        eff = len(passed) / len(total) * 100
        effs.append(eff)
    print("End get_effs()")
    return np.array(effs)
'''
import multiprocessing as mp

def calculate_eff(tree, roll_name):
    print(f'Get eff at {roll_name}')
    roll_mask = (tree['roll_name'] == roll_name)
    total = tree['is_matched'][roll_mask]
    passed = total[total]
    eff = len(passed) / len(total) * 100 if len(total) > 0 else 0
    return eff

def get_effs(data):
    print("Call get_effs()")
    tree = data.tree
    roll_names = np.unique(tree['roll_name'])
    
    # Create a multiprocessing pool
    with mp.Pool(128) as pool:
        # Distribute the work across the pool
        effs = pool.starmap(calculate_eff, [(tree, roll_name) for roll_name in roll_names])
    
    print("End get_effs()")
    return np.array(effs)

def eff_hist(data1, data2, output_path):
    effs1 = get_effs(data1)
    effs2 = get_effs(data2)

    fig, ax = init_figure(
        figsize = (12, 8),
        fontsize = 24,
        com = 13.6,
        label1 = 'Work in Progress',
        label2 = f'{data1.region}',
        loc = 0,
        xlabel = 'Efficiency [%]',
        ylabel = 'Number of Rolls',
        xlim = (70, 100),
        ylim = None,
        xticks = None,
        yticks = None,
        log_scale = False,
    )

    count1, bins1, patch1 = ax.hist(
        effs1, 
        bins = 101, 
        range = (0, 101),
        facecolor = data1.facecolors[0],
        edgecolor = data1.edgecolors[0],
        hatch = data1.hatches[0],
        alpha = 0.5,
        align = 'mid',
        density = False,
        linewidth = 1.6,
        histtype = 'stepfilled',
    )

    count2, bins2, patch2 = ax.hist(
        effs2, 
        bins = 101, 
        range = (0, 101),
        facecolor = data2.facecolors[1],
        edgecolor = data2.edgecolors[1],
        hatch = data2.hatches[1],
        alpha = 0.5,
        align = 'mid',
        density = False,
        linewidth = 1.6,
        histtype = 'stepfilled',
    )
    
    extra = Rectangle((0, 0), 0.1, 0.1, fc='w', fill=False, edgecolor='none', linewidth=0)
    header_row = ['', 'Data', 'Mean', 'Mean(>70%)', 'Entries', 'Underflow']
    data1_row = [
        '', 
        'Run2022', 
        f'{np.mean(effs1) : .2f}', 
        f'{np.mean(effs1[effs1 > 70]) : .2f}',
        f'{len(effs1)}',
        f'{len(effs1[effs1 <= 70])}',
    ]
    data2_row = [
        '', 
        'Run2023', 
        f'{np.mean(effs2) : .2f}', 
        f'{np.mean(effs2[effs2 > 70]) : .2f}',
        f'{len(effs2)}',
        f'{len(effs2[effs2 <= 70])}',
    ]
    legend_handles, legend_values = [], []
    for idx in range(len(header_row)):
        if idx == 0:
            legend_handles += [extra, patch1[0], patch2[0]]
        else:
            legend_handles += [extra, extra, extra]
        legend_values += [header_row[idx], data1_row[idx], data2_row[idx]]

    ax.legend(
        legend_handles, legend_values,
        ncol = len(header_row), columnspacing = 0.0,
        handletextpad = 0.0, handlelength = 1.0,
        alignment = 'center', loc = 'upper left',
        fontsize = 20,
    )
    output_path = Path(output_path)
    if not output_path.parent.exists():
        output_path.parent.mkdir(parents=True)
    fig.savefig(output_path)
    plt.close(fig)
    return [effs1, effs2]


: 

In [ ]:
workspace = Path('/users/hep/eigen1907/Workspace/Workspace-RPC')

input_2022_path = workspace / 'data' / 'efficiency' / 'data' / 'Run2022.root'
roll_blacklist_2022_path = workspace / 'data' / 'efficiency' / 'blacklist' / 'roll-blacklist-2022.json'

input_2023_path = workspace / 'data' / 'efficiency' / 'data' / 'Run2023.root'
roll_blacklist_2023_path = workspace / 'data' / 'efficiency' / 'blacklist' / 'roll-blacklist-2023.json'

run_blacklist_path = workspace / 'data' / 'efficiency' / 'blacklist' / 'run-blacklist.json'
geom_path = workspace / 'data' / 'efficiency' / 'geometry' / 'run3.csv'

run_info = pd.read_csv("/users/hep/eigen1907/Workspace/Workspace-RPC/data/efficiency/run_info/run_info.csv", index_col=False)

data_2022 = DataLoader(
    input_path = input_2022_path,
    geom_path = geom_path,
    roll_blacklist_path = roll_blacklist_2022_path,
    run_blacklist_path = run_blacklist_path,
    var = ['run']
)

data_2023 = DataLoader(
    input_path = input_2023_path,
    geom_path = geom_path,
    roll_blacklist_path = roll_blacklist_2023_path,
    run_blacklist_path = run_blacklist_path,
    var = ['run']
)

: 

In [ ]:
fiducial_data_2022 = data_2022.filter_data(keys = ['is_fiducial'])
fiducial_data_2023 = data_2023.filter_data(keys = ['is_fiducial'])

filter_by_roll_data_2022 = data_2022.filter_data(keys = ['is_fiducial', 'is_linked'])
filter_by_roll_data_2023 = data_2023.filter_data(keys = ['is_fiducial', 'is_linked'])

filter_by_roll_run_data_2022 = data_2022.filter_data(keys = ['is_fiducial', 'is_linked', 'is_safetime'])
filter_by_roll_run_data_2023 = data_2023.filter_data(keys = ['is_fiducial', 'is_linked', 'is_safetime'])

: 

In [ ]:
regions = [
    'All', 'Barrel', 'Endcap', 'Disk123', 'Disk4',
    'W-2', 'W-1', 'W+0', 'W+1', 'W+2',
    'RE+1', 'RE+2', 'RE+3', 'RE+4',
    'RE-1', 'RE-2', 'RE-3', 'RE-4',
]

workspace = Path('/users/hep/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/Plotting/Hist-def-eff/')

for region in regions:
    eff_hist(
        data1 = fiducial_data_2022.filter_data(region = region),
        data2 = fiducial_data_2023.filter_data(region = region),
        output_path = workspace / 'no_filter' / f"{region}.png"
    )
    break
    eff_hist(
        data1 = filter_by_roll_data_2022.filter_data(region = region),
        data2 = filter_by_roll_data_2023.filter_data(region = region),
        output_path = workspace / 'filter_by_roll' / f"{region}.png"
    )

    eff_hist(
        data1 = filter_by_roll_run_data_2022.filter_data(region = region),
        data2 = filter_by_roll_run_data_2023.filter_data(region = region),
        output_path = workspace / 'filter_by_roll_run' / f"{region}.png"
    )


: 